# Tutorial y algoritmo baseline

En este notebook se realizará el algoritmo base a partir del cual comparar el resto. Adicionalmente se muestra como loggear los parámetros de los modelos en usando mlflow 

In [18]:
import sys

MODULE_FULL_PATH = "../"

sys.path.insert(1, MODULE_FULL_PATH)

from settings import IMG_HEIGHT,IMG_WIDTH,BATCH_SIZE,DATA_DIR,SEED
import tensorflow as tf 

TRAIN_DATA_DIR = "../data/train"

dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    batch_size = BATCH_SIZE
)
dataset = dataset.shuffle(9)

Found 10266 files belonging to 29 classes.
Using 8213 files for training.


# Modelo base

El modelo base van a ser 3 capas convulacionales + 1 capa FC y por último la capa de salida.

In [21]:
num_classes = 29

model: tf.keras.Sequential = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,3)),
  tf.keras.layers.Conv2D(16, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 64)       

In [22]:
import mlflow

mlflow.set_experiment("jh")
mlflow.tensorflow.autolog()

with mlflow.start_run():
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    epochs = 10
    history = model.fit(dataset,  callbacks=[callback])


2022-02-21 12:53:02.566406: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100933632 exceeds 10% of free system memory.


  1/257 [..............................] - ETA: 5:46 - loss: 3.3693 - accuracy: 0.0312

2022-02-21 12:53:03.074553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100933632 exceeds 10% of free system memory.


 16/257 [>.............................] - ETA: 2:45 - loss: 3.2325 - accuracy: 0.0586

KeyboardInterrupt: 